![Lancaster University](https://www.lancaster.ac.uk/media/lancaster-university/content-assets/images/fst/logos/SCC-Logo.svg)

# SCC.454: Large Scale Platforms for AI and Data Analysis
## Lab 3: Data Preprocessing with Apache Spark

**Duration:** 2 hours

**Learning Objectives:**
- Understand the importance of data preprocessing in big data pipelines
- Master techniques for handling missing values in Spark DataFrames
- Learn data type conversions and schema management
- Apply text preprocessing techniques at scale
- Use regular expressions (regex) for pattern matching and data extraction
- Build complete data cleaning pipelines in PySpark

## Table of Contents

1. **Part 1: Introduction to Data Preprocessing** (15 minutes)
   - Why Data Preprocessing Matters
   - Setting up the Environment
   - Overview of Spark's Data Cleaning Capabilities

2. **Part 2: Handling Missing Values** (30 minutes)
   - Identifying Missing Data
   - Strategies for Missing Value Treatment
   - Dropping Null Values
   - Filling Missing Values (Imputation)
   - Advanced Imputation Techniques
   - Practical Exercise: Customer Data Cleaning

3. **Part 3: Data Type Handling and Schema Management** (20 minutes)
   - Understanding Spark Data Types
   - Type Casting and Conversion
   - Handling Date and Timestamp Fields
   - Schema Validation and Enforcement

4. **Part 4: Text Preprocessing** (25 minutes)
   - String Functions in Spark
   - Case Normalization and Trimming
   - Tokenization and Text Splitting
   - Stop Word Removal
   - Text Cleaning Pipeline
   - Practical Exercise: Product Review Cleaning

5. **Part 5: Regular Expressions in Spark** (30 minutes)
   - Introduction to Regex in PySpark
   - Pattern Matching with rlike
   - Extracting Data with regexp_extract
   - Replacing Patterns with regexp_replace
   - Complex Pattern Matching
   - Practical Exercise: Log File Parsing

6. **Part 6: Building Complete Preprocessing Pipelines** (15 minutes)
   - Combining Multiple Preprocessing Steps
   - Creating Reusable Cleaning Functions
   - Data Quality Reporting
   - Final Challenge: End-to-End Data Cleaning

---
# Part 1: Introduction to Data Preprocessing
---

## 1.1 Why Data Preprocessing Matters

Data preprocessing is the foundation of any successful data analysis or machine learning project. In the context of big data, preprocessing becomes even more critical due to:

**The Scale Challenge:**
- Real-world datasets often contain millions or billions of records
- Traditional preprocessing tools (like pandas) may not scale effectively
- Distributed processing is essential for handling large volumes

**Common Data Quality Issues:**
- Missing values (null, NaN, empty strings)
- Inconsistent formatting (dates, currencies, units)
- Duplicate records
- Invalid or out-of-range values
- Inconsistent text data (case, whitespace, special characters)
- Unstructured or semi-structured data requiring parsing

**The Garbage In, Garbage Out Principle:**
```
Raw Data (Messy) --> Preprocessing Pipeline --> Clean Data (Analysis-Ready)
```

**Apache Spark's Advantages for Preprocessing:**
- Distributed processing across clusters
- Lazy evaluation for optimization
- Rich built-in functions for data manipulation
- SQL interface for familiar syntax
- Seamless integration with ML pipelines

## 1.2 Setting up the Environment

Let's set up PySpark in Google Colab. If you're running this on a different system, ensure Java is installed and JAVA_HOME is configured correctly.

In [1]:
# Install PySpark
!pip install pyspark==3.5.0 -q

# Install Java (Spark requires Java)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Set Java environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("PySpark and Java installed successfully!")

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
PySpark and Java installed successfully!


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Create a SparkSession configured for data preprocessing
spark = SparkSession.builder \
    .appName("SCC454-DataPreprocessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

# Get the underlying SparkContext
sc = spark.sparkContext

print(f"Spark Version: {spark.version}")
print(f"Application Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")
print("\nSpark Session ready for data preprocessing!")

Spark Version: 4.0.1
Application Name: SCC454-DataPreprocessing
Master: local[*]

Spark Session ready for data preprocessing!


## 1.3 Overview of Spark's Data Cleaning Capabilities

Spark provides several modules and functions for data preprocessing:

**Key Modules:**
- `pyspark.sql.functions`: Built-in functions for transformations
- `pyspark.sql.types`: Data type definitions
- `pyspark.ml.feature`: Feature engineering tools

**Important Function Categories:**

| Category | Functions |
|----------|----------|
| Null Handling | `isNull()`, `isNotNull()`, `na.drop()`, `na.fill()` |
| String Functions | `lower()`, `upper()`, `trim()`, `split()`, `concat()` |
| Regex Functions | `regexp_extract()`, `regexp_replace()`, `rlike()` |
| Type Conversion | `cast()`, `to_date()`, `to_timestamp()` |
| Aggregation | `count()`, `avg()`, `sum()`, `mean()` |
| Conditional | `when()`, `otherwise()`, `coalesce()` |

In [2]:
from pyspark.sql import functions as f

In [3]:
# Import commonly used functions for data preprocessing
from pyspark.sql.functions import (
    col, lit, isnan, isnull, coalesce,
    lower, upper, trim, ltrim, rtrim, length,
    split, concat, concat_ws, substring, replace,
    regexp_extract, regexp_replace,
    to_date, to_timestamp,
    count, avg, sum as spark_sum, mean, stddev,
    min as spark_min, max as spark_max,
    when, expr,
    year, month, dayofmonth, datediff, date_format,
    monotonically_increasing_id, round as spark_round
)

from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType,
    FloatType, DoubleType, DateType, TimestampType, BooleanType
)

print("All preprocessing functions imported successfully!")

All preprocessing functions imported successfully!


---
# Part 2: Handling Missing Values
---

## 2.1 Identifying Missing Data

Missing data is one of the most common data quality issues. In Spark, missing values can appear as:
- `null`: The absence of a value
- `NaN` (Not a Number): Result of undefined numerical operations
- Empty strings: For string columns
- Placeholder values (e.g., -999, "N/A", "Unknown")

Let's create a sample dataset with various types of missing values.

In [3]:
# Create a DataFrame with various missing value patterns
customer_data = [
    (1, "John Smith", 28, "john@email.com", 75000.0, "New York", "2023-01-15"),
    (2, "Jane Doe", None, "jane@email.com", 82000.0, "Los Angeles", "2023-02-20"),
    (3, "Bob Wilson", 35, None, 65000.0, "Chicago", "2023-03-10"),
    (4, "Alice Brown", 42, "alice@email.com", None, "Houston", None),
    (5, None, 31, "unknown@email.com", 70000.0, "Phoenix", "2023-05-01"),
    (6, "Charlie Lee", 29, "", 58000.0, "", "2023-06-15"),
    (7, "Diana Prince", None, "diana@email.com", 95000.0, None, "2023-07-20"),
    (8, "Edward Kim", 38, "N/A", float('nan'), "Boston", "2023-08-25"),
    (9, "Fiona Garcia", 45, "fiona@email.com", 88000.0, "Seattle", "2023-09-30"),
    (10, "George Hall", None, None, None, None, None),
    (11, "Hannah White", 33, "hannah@email.com", 72000.0, "Denver", "2023-11-10"),
    (12, "Ivan Torres", 27, "ivan@email.com", 63000.0, "Miami", "2023-12-05")
]

columns = ["customer_id", "name", "age", "email", "salary", "city", "signup_date"]
df_customers = spark.createDataFrame(customer_data, columns)

print("Customer Data with Missing Values:")
df_customers.show(truncate=False)

Customer Data with Missing Values:
+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|name        |age |email            |salary |city       |signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|1          |John Smith  |28  |john@email.com   |75000.0|New York   |2023-01-15 |
|2          |Jane Doe    |NULL|jane@email.com   |82000.0|Los Angeles|2023-02-20 |
|3          |Bob Wilson  |35  |NULL             |65000.0|Chicago    |2023-03-10 |
|4          |Alice Brown |42  |alice@email.com  |NULL   |Houston    |NULL       |
|5          |NULL        |31  |unknown@email.com|70000.0|Phoenix    |2023-05-01 |
|6          |Charlie Lee |29  |                 |58000.0|           |2023-06-15 |
|7          |Diana Prince|NULL|diana@email.com  |95000.0|NULL       |2023-07-20 |
|8          |Edward Kim  |38  |N/A              |NaN    |Boston     |2023-08-25 |
|9          |Fiona Garcia|45  |fiona@email.com  |88000.0|Seattl

In [4]:
# Check the schema
print("Schema:")
df_customers.printSchema()

Schema:
root
 |-- customer_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- email: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- city: string (nullable = true)
 |-- signup_date: string (nullable = true)



### Counting Missing Values

The first step in handling missing data is to understand its extent.

In [6]:
# Method 1: Count nulls per column
print("Null counts per column:")
null_counts = df_customers.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_customers.columns
])
null_counts.show()

Null counts per column:
+-----------+----+---+-----+------+----+-----------+
|customer_id|name|age|email|salary|city|signup_date|
+-----------+----+---+-----+------+----+-----------+
|          0|   1|  3|    2|     2|   2|          2|
+-----------+----+---+-----+------+----+-----------+



In [ ]:
# Method 2: Count nulls AND NaN values (important for numeric columns)
print("Null + NaN counts for numeric columns:")
df_customers.select([
    f.count(f.when(f.col("salary").isNull() | f.isnan(f.col("salary")), 1)).alias("salary_missing")
]).show()

Null + NaN counts for numeric columns:
+--------------+
|salary_missing|
+--------------+
|             3|
+--------------+



In [7]:
df_customers.show()

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|  28|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|NULL|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|  35|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|  42|  alice@email.com|   NULL|    Houston|       NULL|
|          5|        NULL|  31|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|  29|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|NULL|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|  38|              N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|  45|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

In [10]:
# Method 3: Comprehensive missing value report
def missing_value_report(df):
    total_rows = df.count()
    report_data = []
    for col_name in df.columns:
        col_type = str(df.schema[col_name].dataType)
        null_count = df.filter(col(col_name).isNull()).count()
        empty_count = 0
        if "String" in col_type:
            empty_count = df.filter(
                (col(col_name) == "") | (col(col_name) == "N/A") | (col(col_name) == "Unknown")
            ).count()
        nan_count = 0
        if "Double" in col_type or "Float" in col_type:
            nan_count = df.filter(isnan(col(col_name))).count()
        total_missing = null_count + empty_count + nan_count
        missing_pct = (total_missing / total_rows) * 100
        report_data.append((col_name, col_type, null_count, empty_count, nan_count, total_missing, round(missing_pct, 2)))
    report_columns = ["Column", "Type", "Nulls", "Empty", "NaN", "Total_Missing", "Missing_%"]
    return spark.createDataFrame(report_data, report_columns)

print("="*60)
print("COMPREHENSIVE MISSING VALUE REPORT")
print("="*60)
report = missing_value_report(df_customers)
report.show(truncate=False)

COMPREHENSIVE MISSING VALUE REPORT
+-----------+------------+-----+-----+---+-------------+---------+
|Column     |Type        |Nulls|Empty|NaN|Total_Missing|Missing_%|
+-----------+------------+-----+-----+---+-------------+---------+
|customer_id|LongType()  |0    |0    |0  |0            |0.0      |
|name       |StringType()|1    |0    |0  |1            |8.33     |
|age        |LongType()  |3    |0    |0  |3            |25.0     |
|email      |StringType()|2    |2    |0  |4            |33.33    |
|salary     |DoubleType()|2    |0    |1  |3            |25.0     |
|city       |StringType()|2    |1    |0  |3            |25.0     |
|signup_date|StringType()|2    |0    |0  |2            |16.67    |
+-----------+------------+-----+-----+---+-------------+---------+



## 2.2 Strategies for Missing Value Treatment

There are several strategies for handling missing values:

1. **Deletion**: Remove rows or columns with missing values
2. **Imputation**: Fill missing values with estimated values (mean, median, mode)
3. **Flagging**: Create indicator variables for missingness
4. **Keep as-is**: Leave missing values for algorithms that can handle them

## 2.3 Dropping Null Values

Spark's `na.drop()` method provides flexible options for removing rows with missing values.

In [10]:
# Original count
print(f"Original row count: {df_customers.count()}")

# Drop rows with ANY null value (most aggressive)
df_drop_any = df_customers.na.drop(how="any")
print(f"After dropping rows with ANY null: {df_drop_any.count()}")
df_drop_any.show()

Original row count: 12
After dropping rows with ANY null: 5
+-----------+------------+---+----------------+-------+--------+-----------+
|customer_id|        name|age|           email| salary|    city|signup_date|
+-----------+------------+---+----------------+-------+--------+-----------+
|          1|  John Smith| 28|  john@email.com|75000.0|New York| 2023-01-15|
|          6| Charlie Lee| 29|                |58000.0|        | 2023-06-15|
|          9|Fiona Garcia| 45| fiona@email.com|88000.0| Seattle| 2023-09-30|
|         11|Hannah White| 33|hannah@email.com|72000.0|  Denver| 2023-11-10|
|         12| Ivan Torres| 27|  ivan@email.com|63000.0|   Miami| 2023-12-05|
+-----------+------------+---+----------------+-------+--------+-----------+



In [11]:
# Drop rows where ALL values are null (least aggressive)
df_drop_all = df_customers.na.drop(how="all")
print(f"After dropping rows where ALL are null: {df_drop_all.count()}")

After dropping rows where ALL are null: 12


In [12]:
# Drop rows where specific columns are null (most practical)
df_drop_subset = df_customers.na.drop(subset=["name", "email"])
print(f"After dropping rows where name OR email is null: {df_drop_subset.count()}")
df_drop_subset.show()

After dropping rows where name OR email is null: 9
+-----------+------------+----+----------------+-------+-----------+-----------+
|customer_id|        name| age|           email| salary|       city|signup_date|
+-----------+------------+----+----------------+-------+-----------+-----------+
|          1|  John Smith|  28|  john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|NULL|  jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          4| Alice Brown|  42| alice@email.com|   NULL|    Houston|       NULL|
|          6| Charlie Lee|  29|                |58000.0|           | 2023-06-15|
|          7|Diana Prince|NULL| diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|  38|             N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|  45| fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         11|Hannah White|  33|hannah@email.com|72000.0|     Denver| 2023-11-10|
|         12| Ivan Torres|  27|  ivan@email.com|63000.0|  

In [ ]:
# Drop rows with fewer than a threshold of non-null values
df_drop_thresh = df_customers.na.drop(thresh=5)
print(f"After dropping rows with < 5 non-null values: {df_drop_thresh.count()}")
df_drop_thresh.show()

## 2.4 Filling Missing Values (Imputation)

Spark's `na.fill()` method allows filling missing values with specified values.

In [11]:
# Fill all numeric columns with a single value
print("Fill all numeric nulls with 0:")
df_customers.na.fill(0).show()

Fill all numeric nulls with 0:
+-----------+------------+---+-----------------+-------+-----------+-----------+
|customer_id|        name|age|            email| salary|       city|signup_date|
+-----------+------------+---+-----------------+-------+-----------+-----------+
|          1|  John Smith| 28|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|  0|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson| 35|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown| 42|  alice@email.com|    0.0|    Houston|       NULL|
|          5|        NULL| 31|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee| 29|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|  0|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim| 38|              N/A|    0.0|     Boston| 2023-08-25|
|          9|Fiona Garcia| 45|  fiona@email.com|88000.0|    Seattle| 2023-09-3

In [12]:
# Fill specific columns with specific values (dictionary approach)
fill_values = {
    "name": "Unknown",
    "age": 30,
    "email": "no_email@placeholder.com",
    "salary": 0.0,
    "city": "Unknown",
    "signup_date": "1970-01-01"
}

print("Fill with specific values per column:")
df_filled = df_customers.na.fill(fill_values)
df_filled.show(truncate=False)

Fill with specific values per column:
+-----------+------------+---+------------------------+-------+-----------+-----------+
|customer_id|name        |age|email                   |salary |city       |signup_date|
+-----------+------------+---+------------------------+-------+-----------+-----------+
|1          |John Smith  |28 |john@email.com          |75000.0|New York   |2023-01-15 |
|2          |Jane Doe    |30 |jane@email.com          |82000.0|Los Angeles|2023-02-20 |
|3          |Bob Wilson  |35 |no_email@placeholder.com|65000.0|Chicago    |2023-03-10 |
|4          |Alice Brown |42 |alice@email.com         |0.0    |Houston    |1970-01-01 |
|5          |Unknown     |31 |unknown@email.com       |70000.0|Phoenix    |2023-05-01 |
|6          |Charlie Lee |29 |                        |58000.0|           |2023-06-15 |
|7          |Diana Prince|30 |diana@email.com         |95000.0|Unknown    |2023-07-20 |
|8          |Edward Kim  |38 |N/A                     |0.0    |Boston     |2023-08

### Statistical Imputation

For numerical columns, it's often better to use statistical measures like mean or median.

In [13]:
# Calculate mean for age column (excluding nulls)
age_stats = df_customers.select(
    mean(col("age")).alias("mean_age"),
    expr("percentile_approx(age, 0.5)").alias("median_age")
).collect()[0]

mean_age = age_stats["mean_age"]
median_age = age_stats["median_age"]

print(f"Mean age: {mean_age:.2f}")
print(f"Median age: {median_age}")

# Fill with mean
df_mean_imputed = df_customers.na.fill({"age": int(mean_age)})
print("\nAge filled with mean:")
df_mean_imputed.select("customer_id", "name", "age").show()

Mean age: 34.22
Median age: 33

Age filled with mean:
+-----------+------------+---+
|customer_id|        name|age|
+-----------+------------+---+
|          1|  John Smith| 28|
|          2|    Jane Doe| 34|
|          3|  Bob Wilson| 35|
|          4| Alice Brown| 42|
|          5|        NULL| 31|
|          6| Charlie Lee| 29|
|          7|Diana Prince| 34|
|          8|  Edward Kim| 38|
|          9|Fiona Garcia| 45|
|         10| George Hall| 34|
|         11|Hannah White| 33|
|         12| Ivan Torres| 27|
+-----------+------------+---+



In [27]:
# Calculate and impute with median for salary
salary_stats = df_customers.filter(
    col("salary").isNotNull() & ~isnan(col("salary"))
).select(
    mean(col("salary")).alias("mean_salary"),
    expr("percentile_approx(salary, 0.5)").alias("median_salary")
).collect()[0]

print(f"Mean salary: ${salary_stats['mean_salary']:,.2f}")
print(f"Median salary: ${salary_stats['median_salary']:,.2f}")

Mean salary: $74,222.22
Median salary: $72,000.00


## 2.5 Advanced Imputation Techniques

In [29]:
# # Group-wise imputation: Fill missing values based on group statistics
from pyspark.sql.window import Window

# Calculate average salary per city
city_avg_salary = df_customers.filter(
    col("salary").isNotNull() & ~isnan(col("salary")) & col("city").isNotNull()
).groupBy("city").agg(
    spark_round(mean("salary"), 2).alias("city_avg_salary")
)

print("Average salary per city:")
city_avg_salary.show()

Average salary per city:
+-----------+---------------+
|       city|city_avg_salary|
+-----------+---------------+
|    Phoenix|        70000.0|
|Los Angeles|        82000.0|
|    Chicago|        65000.0|
|           |        58000.0|
|   New York|        75000.0|
|    Seattle|        88000.0|
|      Miami|        63000.0|
|     Denver|        72000.0|
+-----------+---------------+



In [ ]:
# Using coalesce for conditional filling
overall_avg_salary = salary_stats['mean_salary']

df_smart_imputed = df_customers.join(
    city_avg_salary, on="city", how="left"
).withColumn(
    "salary_imputed",
    coalesce(
        when(~isnan(col("salary")), col("salary")),
        col("city_avg_salary"),
        lit(overall_avg_salary)
    )
).select(
    "customer_id", "name", "city", "salary", "city_avg_salary", "salary_imputed"
)

print("Smart imputation using city averages:")
df_smart_imputed.show()

Smart imputation using city averages:
+-----------+------------+-----------+-------+---------------+-----------------+
|customer_id|        name|       city| salary|city_avg_salary|   salary_imputed|
+-----------+------------+-----------+-------+---------------+-----------------+
|          2|    Jane Doe|Los Angeles|82000.0|        82000.0|          82000.0|
|          3|  Bob Wilson|    Chicago|65000.0|        65000.0|          65000.0|
|          1|  John Smith|   New York|75000.0|        75000.0|          75000.0|
|          5|        NULL|    Phoenix|70000.0|        70000.0|          70000.0|
|          4| Alice Brown|    Houston|   NULL|           NULL|74222.22222222222|
|          6| Charlie Lee|           |58000.0|        58000.0|          58000.0|
|          7|Diana Prince|       NULL|95000.0|           NULL|          95000.0|
|          9|Fiona Garcia|    Seattle|88000.0|        88000.0|          88000.0|
|          8|  Edward Kim|     Boston|    NaN|           NULL|74222.222

In [30]:
# Forward Fill using Window functions (useful for time-series)
from pyspark.sql.functions import last, first

ts_data = [
    ("2024-01-01", 100.0),
    ("2024-01-02", None),
    ("2024-01-03", None),
    ("2024-01-04", 105.0),
    ("2024-01-05", None),
    ("2024-01-06", 108.0),
    ("2024-01-07", None),
]

df_ts = spark.createDataFrame(ts_data, ["date", "value"])
print("Time series with missing values:")
df_ts.show()

# Forward fill
window_forward = Window.orderBy("date").rowsBetween(Window.unboundedPreceding, 0)

df_forward_fill = df_ts.withColumn(
    "value_ffill",
    last(col("value"), ignorenulls=True).over(window_forward)
)

print("After forward fill:")
df_forward_fill.show()

Time series with missing values:
+----------+-----+
|      date|value|
+----------+-----+
|2024-01-01|100.0|
|2024-01-02| NULL|
|2024-01-03| NULL|
|2024-01-04|105.0|
|2024-01-05| NULL|
|2024-01-06|108.0|
|2024-01-07| NULL|
+----------+-----+

After forward fill:
+----------+-----+-----------+
|      date|value|value_ffill|
+----------+-----+-----------+
|2024-01-01|100.0|      100.0|
|2024-01-02| NULL|      100.0|
|2024-01-03| NULL|      100.0|
|2024-01-04|105.0|      105.0|
|2024-01-05| NULL|      105.0|
|2024-01-06|108.0|      108.0|
|2024-01-07| NULL|      108.0|
+----------+-----+-----------+



In [21]:
# Creating missing value indicator columns
df_with_indicators = df_customers.withColumn(
    "age_was_missing",
    when(col("age").isNull(), 1).otherwise(0)
).withColumn(
    "salary_was_missing",
    when(col("salary").isNull() | isnan(col("salary")), 1).otherwise(0)
)

print("Data with missing value indicators:")
df_with_indicators.select(
    "customer_id", "name", "age", "age_was_missing", "salary", "salary_was_missing"
).show()

Data with missing value indicators:
+-----------+------------+----+---------------+-------+------------------+
|customer_id|        name| age|age_was_missing| salary|salary_was_missing|
+-----------+------------+----+---------------+-------+------------------+
|          1|  John Smith|  28|              0|75000.0|                 0|
|          2|    Jane Doe|NULL|              1|82000.0|                 0|
|          3|  Bob Wilson|  35|              0|65000.0|                 0|
|          4| Alice Brown|  42|              0|   NULL|                 1|
|          5|        NULL|  31|              0|70000.0|                 0|
|          6| Charlie Lee|  29|              0|58000.0|                 0|
|          7|Diana Prince|NULL|              1|95000.0|                 0|
|          8|  Edward Kim|  38|              0|    NaN|                 1|
|          9|Fiona Garcia|  45|              0|88000.0|                 0|
|         10| George Hall|NULL|              1|   NULL|         

## 2.6 Practical Exercise: Customer Data Cleaning

Now it's your turn! Clean the customer dataset by applying appropriate imputation strategies.

**Tasks:**
1. Handle missing names (fill with "Customer_[ID]")
2. Impute missing ages with the median age
3. Replace empty/placeholder emails with a standardized format
4. Impute missing salaries with the median salary
5. Fill missing cities with "Unknown"
6. Handle missing signup dates appropriately
7. Remove the NaN value from salary column
8. Create a data quality score

In [31]:
# Your code here
df_customers.show()
print(df_customers.columns)

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|  28|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|NULL|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|  35|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|  42|  alice@email.com|   NULL|    Houston|       NULL|
|          5|        NULL|  31|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|  29|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|NULL|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|  38|              N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|  45|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

In [34]:
##find amount of missing values in the dataframe
from pyspark.sql.functions import isnan,when,count,col,isnull
##first find the number only

df_customers.select(
   [
    count(
        when(
            isnull(c) | 
            (isnan(col(c)) if t in ("double", "float", "int") else lit(False)) |
            ((col(c) == "") if t == "string" else lit(False)) |
            ((col(c).contains("N/A")) if t == "string" else lit(False)), 
            c
        )
    ).alias(c) 
    for c, t in df_customers.dtypes
]).show()

+-----------+----+---+-----+------+----+-----------+
|customer_id|name|age|email|salary|city|signup_date|
+-----------+----+---+-----+------+----+-----------+
|          0|   1|  3|    4|     3|   3|          2|
+-----------+----+---+-----+------+----+-----------+



replace names with customer ID

In [36]:
df_customers_cleaned = df_customers.withColumn(
    'name',
    when(col('name').isNull(),col('customer_id').cast("string")).otherwise(col('name'))
)

df_customers_cleaned.show()

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|  28|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|NULL|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|  35|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|  42|  alice@email.com|   NULL|    Houston|       NULL|
|          5|           5|  31|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|  29|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|NULL|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|  38|              N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|  45|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

### IMPUTE MISSING AGES WITH MEDIAN

In [37]:
from pyspark.sql.functions import median

In [38]:
df_customers_cleaned = df_customers_cleaned.withColumn(
    'age',
    when(col('age').isNull(),df_customers.approxQuantile("age", [0.5], 0.01)[0]).otherwise(col("age")))

df_customers_cleaned.show()

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|28.0|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|33.0|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|35.0|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|42.0|  alice@email.com|   NULL|    Houston|       NULL|
|          5|           5|31.0|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|29.0|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|33.0|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|38.0|              N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|45.0|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

replace empty emails with a standardized format

In [96]:
df_customers_cleaned = df_customers_cleaned.withColumn(
    'email', when((col('email').isNull()) | (col('email') == "") | (col('email').contains("N/A")),"random@email.com").otherwise(col('email')))

df_customers_cleaned.show()


+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|28.0|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|33.0|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|35.0| random@email.com|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|42.0|  alice@email.com|72000.0|    Houston|       NULL|
|          5|           5|31.0|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|29.0| random@email.com|58000.0|           | 2023-06-15|
|          7|Diana Prince|33.0|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|38.0| random@email.com|72000.0|     Boston| 2023-08-25|
|          9|Fiona Garcia|45.0|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

Replacing Salary

In [97]:
df_customers_cleaned = df_customers_cleaned.withColumn(
    'salary',
    when((col('salary').isNull() ) | (isnan(col="salary")),df_customers.approxQuantile("salary", [0.5], 0.01)[0]).otherwise(col("salary")))

df_customers_cleaned.show()

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|28.0|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|33.0|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|35.0| random@email.com|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|42.0|  alice@email.com|72000.0|    Houston|       NULL|
|          5|           5|31.0|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|29.0| random@email.com|58000.0|           | 2023-06-15|
|          7|Diana Prince|33.0|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|38.0| random@email.com|72000.0|     Boston| 2023-08-25|
|          9|Fiona Garcia|45.0|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

Handling missing city

In [ ]:
df_customers_cleaned = df_customers_cleaned.withColumn(
    'city',
    when((col('city').isNull()) | (col('city') == ""), "Unknown").otherwise(col("city")))

df_customers_cleaned.show()

---
# Part 3: Data Type Handling and Schema Management
---

## 3.1 Understanding Spark Data Types

Proper data types are essential for efficient processing.

**Numeric Types:** ByteType, ShortType, IntegerType, LongType, FloatType, DoubleType, DecimalType

**String and Binary:** StringType, BinaryType

**Date/Time:** DateType, TimestampType

**Complex Types:** ArrayType, MapType, StructType

In [39]:
# Create a DataFrame with mixed type issues
messy_data = [
    ("1", "25", "50000.50", "2024-01-15", "true"),
    ("2", "thirty", "60000", "15/02/2024", "yes"),
    ("3", "35", "$75,000", "2024-03-20", "1"),
    ("4", "40", "80000.00", "March 25, 2024", "false"),
    ("5", "N/A", "invalid", "2024-04-30", "no"),
]

df_messy = spark.createDataFrame(messy_data, ["id", "age", "salary", "date", "active"])

print("Messy data with type issues:")
df_messy.show()
df_messy.printSchema()

Messy data with type issues:
+---+------+--------+--------------+------+
| id|   age|  salary|          date|active|
+---+------+--------+--------------+------+
|  1|    25|50000.50|    2024-01-15|  true|
|  2|thirty|   60000|    15/02/2024|   yes|
|  3|    35| $75,000|    2024-03-20|     1|
|  4|    40|80000.00|March 25, 2024| false|
|  5|   N/A| invalid|    2024-04-30|    no|
+---+------+--------+--------------+------+

root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active: string (nullable = true)



## 3.2 Type Casting and Conversion

In [40]:
# Simple casting
df_cast = df_messy.withColumn("id_int", col("id").cast(IntegerType()))
print("ID cast to integer:")
df_cast.select("id", "id_int").show()

ID cast to integer:
+---+------+
| id|id_int|
+---+------+
|  1|     1|
|  2|     2|
|  3|     3|
|  4|     4|
|  5|     5|
+---+------+



In [41]:
# Casting with potential failures - non-numeric strings become null
df_age_cast = df_messy.withColumn("age_int", col("age").cast(IntegerType()))
print("Age cast to integer (note nulls for invalid values):")
df_age_cast.select("age", "age_int").show()

Age cast to integer (note nulls for invalid values):


{"ts": "2026-02-06 18:27:40.989", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[CAST_INVALID_INPUT] The value 'thirty' of the type \"STRING\" cannot be cast to \"INT\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018", "context": {"file": "line 2 in cell [41]", "line": "", "fragment": "cast", "errorClass": "CAST_INVALID_INPUT"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o997.showString.\n: org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'thirty' of the type \"STRING\" cannot be cast to \"INT\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018\n== DataFrame ==\n\"cast\" was called from\nline 2 in cell [41]\n\n\tat org.apache.spark.sql.errors.QueryExecutionEr

NumberFormatException: [CAST_INVALID_INPUT] The value 'thirty' of the type "STRING" cannot be cast to "INT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"cast" was called from
line 2 in cell [41]


In [42]:
# Clean salary string before casting
df_salary_clean = df_messy.withColumn(
    "salary_cleaned",
    regexp_replace(col("salary"), "[$,]", "")
).withColumn(
    "salary_double",
    col("salary_cleaned").cast(DoubleType())
)

print("Salary cleaning and casting:")
df_salary_clean.select("salary", "salary_cleaned", "salary_double").show()

Salary cleaning and casting:


{"ts": "2026-02-06 18:28:58.482", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[CAST_INVALID_INPUT] The value 'invalid' of the type \"STRING\" cannot be cast to \"DOUBLE\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018", "context": {"file": "line 7 in cell [42]", "line": "", "fragment": "cast", "errorClass": "CAST_INVALID_INPUT"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o1024.showString.\n: org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'invalid' of the type \"STRING\" cannot be cast to \"DOUBLE\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018\n== DataFrame ==\n\"cast\" was called from\nline 7 in cell [42]\n\n\tat org.apache.spark.sql.errors.QueryEx

NumberFormatException: [CAST_INVALID_INPUT] The value 'invalid' of the type "STRING" cannot be cast to "DOUBLE" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"cast" was called from
line 7 in cell [42]


In [ ]:
# Boolean conversion with multiple representations
df_bool = df_messy.withColumn(
    "active_bool",
    when(lower(col("active")).isin("true", "yes", "1", "t", "y"), True)
    .when(lower(col("active")).isin("false", "no", "0", "f", "n"), False)
    .otherwise(None)
)

print("Boolean conversion:")
df_bool.select("active", "active_bool").show()

## 3.3 Handling Date and Timestamp Fields

In [ ]:
# Common date formats
date_examples = [
    ("2024-01-15", "ISO format"),
    ("15/02/2024", "European DD/MM/YYYY"),
    ("03/20/2024", "American MM/DD/YYYY"),
    ("March 25, 2024", "Written format"),
    ("2024.04.30", "Dot separator"),
]

df_dates = spark.createDataFrame(date_examples, ["date_string", "format_name"])
df_dates.show(truncate=False)

In [ ]:
# Parse dates with multiple format patterns using coalesce
df_parsed = df_dates.withColumn(
    "parsed_date",
    coalesce(
        to_date(col("date_string"), "yyyy-MM-dd"),
        to_date(col("date_string"), "dd/MM/yyyy"),
        to_date(col("date_string"), "MM/dd/yyyy"),
        to_date(col("date_string"), "MMMM dd, yyyy"),
        to_date(col("date_string"), "yyyy.MM.dd")
    )
)

print("Dates parsed from multiple formats:")
df_parsed.show(truncate=False)

In [ ]:
# Extract components from dates
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, quarter, date_format

df_date_parts = df_parsed.filter(col("parsed_date").isNotNull()).select(
    col("date_string"),
    col("parsed_date"),
    year(col("parsed_date")).alias("year"),
    month(col("parsed_date")).alias("month"),
    dayofmonth(col("parsed_date")).alias("day"),
    quarter(col("parsed_date")).alias("quarter"),
    date_format(col("parsed_date"), "EEEE").alias("day_name")
)

print("Date components extracted:")
df_date_parts.show(truncate=False)

## 3.4 Schema Validation and Enforcement

In [50]:
# Define an expected schema
expected_schema = StructType([
    StructField("customer_id", IntegerType(), nullable=False),
    StructField("name", StringType(), nullable=True),
    StructField("age", IntegerType(), nullable=True),
    StructField("email", StringType(), nullable=True),
    StructField("salary", DoubleType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("signup_date", DateType(), nullable=True),
    StructField("testing2", StringType(), nullable=True)
])

print("Expected Schema:")
for field in expected_schema.fields:
    print(f"  {field.name}: {field.dataType} (nullable={field.nullable})")

Expected Schema:
  customer_id: IntegerType() (nullable=False)
  name: StringType() (nullable=True)
  age: IntegerType() (nullable=True)
  email: StringType() (nullable=True)
  salary: DoubleType() (nullable=True)
  city: StringType() (nullable=True)
  signup_date: DateType() (nullable=True)
  testing2: StringType() (nullable=True)


In [45]:
df_customers.show()

+-----------+------------+----+-----------------+-------+-----------+-----------+
|customer_id|        name| age|            email| salary|       city|signup_date|
+-----------+------------+----+-----------------+-------+-----------+-----------+
|          1|  John Smith|  28|   john@email.com|75000.0|   New York| 2023-01-15|
|          2|    Jane Doe|NULL|   jane@email.com|82000.0|Los Angeles| 2023-02-20|
|          3|  Bob Wilson|  35|             NULL|65000.0|    Chicago| 2023-03-10|
|          4| Alice Brown|  42|  alice@email.com|   NULL|    Houston|       NULL|
|          5|        NULL|  31|unknown@email.com|70000.0|    Phoenix| 2023-05-01|
|          6| Charlie Lee|  29|                 |58000.0|           | 2023-06-15|
|          7|Diana Prince|NULL|  diana@email.com|95000.0|       NULL| 2023-07-20|
|          8|  Edward Kim|  38|              N/A|    NaN|     Boston| 2023-08-25|
|          9|Fiona Garcia|  45|  fiona@email.com|88000.0|    Seattle| 2023-09-30|
|         10| Ge

In [51]:
# Function to validate and transform data to match expected schema
def enforce_schema(df, target_schema):
    result_df = df
    for field in target_schema.fields:
        col_name = field.name
        col_type = field.dataType
        if col_name in df.columns:
            result_df = result_df.withColumn(col_name, col(col_name).cast(col_type))
        else:
            result_df = result_df.withColumn(col_name, lit("Maaz").cast(col_type))
    return result_df.select([field.name for field in target_schema.fields])

df_enforced = enforce_schema(df_customers, expected_schema)
print("Schema-enforced DataFrame:")
df_enforced.printSchema()
df_enforced.show(5)

Schema-enforced DataFrame:
root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- city: string (nullable = true)
 |-- signup_date: date (nullable = true)
 |-- testing2: string (nullable = false)

+-----------+-----------+----+-----------------+-------+-----------+-----------+--------+
|customer_id|       name| age|            email| salary|       city|signup_date|testing2|
+-----------+-----------+----+-----------------+-------+-----------+-----------+--------+
|          1| John Smith|  28|   john@email.com|75000.0|   New York| 2023-01-15|    Maaz|
|          2|   Jane Doe|NULL|   jane@email.com|82000.0|Los Angeles| 2023-02-20|    Maaz|
|          3| Bob Wilson|  35|             NULL|65000.0|    Chicago| 2023-03-10|    Maaz|
|          4|Alice Brown|  42|  alice@email.com|   NULL|    Houston|       NULL|    Maaz|
|          5|       NULL|  3

---
# Part 4: Text Preprocessing
---

## 4.1 String Functions in Spark

Text data often requires extensive preprocessing before analysis.

In [4]:
# Create sample text data
text_data = [
    (1, "   Hello World   ", "john.doe@email.com"),
    (2, "APACHE SPARK", "JANE.DOE@EMAIL.COM"),
    (3, "data science", "bob_wilson@company.co.uk"),
    (4, "  Machine Learning  ", "alice-brown@domain.org"),
    (5, "NLP is Great!!!", "charlie.lee123@test.com"),
]

df_text = spark.createDataFrame(text_data, ["id", "text", "email"])
print("Sample text data:")
df_text.show(truncate=False)

Sample text data:
+---+--------------------+------------------------+
|id |text                |email                   |
+---+--------------------+------------------------+
|1  |   Hello World      |john.doe@email.com      |
|2  |APACHE SPARK        |JANE.DOE@EMAIL.COM      |
|3  |data science        |bob_wilson@company.co.uk|
|4  |  Machine Learning  |alice-brown@domain.org  |
|5  |NLP is Great!!!     |charlie.lee123@test.com |
+---+--------------------+------------------------+



## 4.2 Case Normalization and Trimming

In [5]:
# Case conversion
df_case = df_text.select(
    col("text"),
    lower(col("text")).alias("lowercase"),
    upper(col("text")).alias("uppercase"),
    expr("initcap(text)").alias("titlecase")
)

print("Case conversion:")
df_case.show(truncate=False)

Case conversion:
+--------------------+--------------------+--------------------+--------------------+
|text                |lowercase           |uppercase           |titlecase           |
+--------------------+--------------------+--------------------+--------------------+
|   Hello World      |   hello world      |   HELLO WORLD      |   Hello World      |
|APACHE SPARK        |apache spark        |APACHE SPARK        |Apache Spark        |
|data science        |data science        |DATA SCIENCE        |Data Science        |
|  Machine Learning  |  machine learning  |  MACHINE LEARNING  |  Machine Learning  |
|NLP is Great!!!     |nlp is great!!!     |NLP IS GREAT!!!     |Nlp Is Great!!!     |
+--------------------+--------------------+--------------------+--------------------+



In [6]:
# Trimming whitespace
df_trim = df_text.select(
    col("text"),
    trim(col("text")).alias("trimmed"),
    ltrim(col("text")).alias("left_trimmed"),
    rtrim(col("text")).alias("right_trimmed"),
    length(col("text")).alias("original_len"),
    length(trim(col("text"))).alias("trimmed_len")
)

print("Whitespace trimming:")
df_trim.show(truncate=False)

Whitespace trimming:
+--------------------+----------------+------------------+------------------+------------+-----------+
|text                |trimmed         |left_trimmed      |right_trimmed     |original_len|trimmed_len|
+--------------------+----------------+------------------+------------------+------------+-----------+
|   Hello World      |Hello World     |Hello World       |   Hello World    |17          |11         |
|APACHE SPARK        |APACHE SPARK    |APACHE SPARK      |APACHE SPARK      |12          |12         |
|data science        |data science    |data science      |data science      |12          |12         |
|  Machine Learning  |Machine Learning|Machine Learning  |  Machine Learning|20          |16         |
|NLP is Great!!!     |NLP is Great!!! |NLP is Great!!!   |NLP is Great!!!   |15          |15         |
+--------------------+----------------+------------------+------------------+------------+-----------+



## 4.3 Tokenization and Text Splitting

In [7]:
# Create sample sentences
sentences_data = [
    (1, "Apache Spark is a unified analytics engine for big data processing"),
    (2, "Natural language processing enables computers to understand text"),
    (3, "Machine learning models require clean preprocessed data"),
    (4, "Text mining extracts valuable insights from unstructured data"),
]

df_sentences = spark.createDataFrame(sentences_data, ["id", "sentence"])
df_sentences.show(truncate=False)

+---+------------------------------------------------------------------+
|id |sentence                                                          |
+---+------------------------------------------------------------------+
|1  |Apache Spark is a unified analytics engine for big data processing|
|2  |Natural language processing enables computers to understand text  |
|3  |Machine learning models require clean preprocessed data           |
|4  |Text mining extracts valuable insights from unstructured data     |
+---+------------------------------------------------------------------+



In [8]:
# Basic tokenization using split
from pyspark.sql.functions import split, size, explode

df_tokens = df_sentences.withColumn(
    "words",
    split(lower(col("sentence")), " ")
).withColumn(
    "word_count",
    size(col("words"))
)

print("Tokenized sentences:")
df_tokens.show(truncate=False)

Tokenized sentences:
+---+------------------------------------------------------------------+------------------------------------------------------------------------------+----------+
|id |sentence                                                          |words                                                                         |word_count|
+---+------------------------------------------------------------------+------------------------------------------------------------------------------+----------+
|1  |Apache Spark is a unified analytics engine for big data processing|[apache, spark, is, a, unified, analytics, engine, for, big, data, processing]|11        |
|2  |Natural language processing enables computers to understand text  |[natural, language, processing, enables, computers, to, understand, text]     |8         |
|3  |Machine learning models require clean preprocessed data           |[machine, learning, models, require, clean, preprocessed, data]               |7         |
|

In [10]:
# Explode tokens and calculate word frequency
df_exploded = df_tokens.select(
    col("id"),
    explode(col("words")).alias("word")
)
# df_exploded.show()
word_freq = df_exploded.groupBy("word").agg(
    count("*").alias("frequency")
).orderBy(col("frequency").desc())

print("Word frequencies:")
word_freq.show(15)

Word frequencies:
+----------+---------+
|      word|frequency|
+----------+---------+
|      data|        3|
|      text|        2|
|processing|        2|
| computers|        1|
|       for|        1|
|   natural|        1|
|        is|        1|
|    apache|        1|
|     spark|        1|
|  language|        1|
| analytics|        1|
|   enables|        1|
|understand|        1|
|         a|        1|
|   unified|        1|
+----------+---------+
only showing top 15 rows


## 4.4 Stop Word Removal

In [17]:
# Define common English stop words
stop_words = [
    "a", "an", "the", "and", "or", "but", "is", "are", "was", "were",
    "to", "of", "in", "for", "on", "with", "at", "by", "from",
    "it", "its", "this", "that", "these", "those",
    "be", "been", "being", "have", "has", "had"
]

from pyspark.sql.functions import array_except, array

#converting into array for spark to use
stop_words_array = array([lit(w) for w in stop_words])

df_no_stopwords = df_tokens.withColumn(
    "words_filtered",
    array_except(col("words"), stop_words_array)
).withColumn(
    "words_filtered_sentence",
    concat_ws(" ",col("words_filtered"))
)

print("Words with stop words removed:")
df_no_stopwords.select("sentence", "words", "words_filtered","words_filtered_sentence").show(truncate=False)

Words with stop words removed:
+------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-------------------------------------------------------------+
|sentence                                                          |words                                                                         |words_filtered                                                       |words_filtered_sentence                                      |
+------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-------------------------------------------------------------+
|Apache Spark is a unified analytics engine for big data processing|[apache, spark, is, a, unified, analytics, engine, for, big, 

In [18]:
# Using Spark ML's StopWordsRemover
from pyspark.ml.feature import StopWordsRemover, Tokenizer

tokenizer = Tokenizer(inputCol="sentence", outputCol="words_ml")
df_tokenized_ml = tokenizer.transform(df_sentences)

remover = StopWordsRemover(inputCol="words_ml", outputCol="words_cleaned")
df_cleaned_ml = remover.transform(df_tokenized_ml)

print("Using Spark ML StopWordsRemover:")
df_cleaned_ml.select("sentence", "words_ml", "words_cleaned").show(truncate=False)

Using Spark ML StopWordsRemover:
+------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+
|sentence                                                          |words_ml                                                                      |words_cleaned                                                        |
+------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+
|Apache Spark is a unified analytics engine for big data processing|[apache, spark, is, a, unified, analytics, engine, for, big, data, processing]|[apache, spark, unified, analytics, engine, big, data, processing]   |
|Natural language processing enables computers to understand text  |[natural, language, process

## 4.5 Text Cleaning Pipeline

In [19]:
# Sample product reviews (messy text data)
reviews_data = [
    (1, "  GREAT product!!!  Would buy again.   ", 5),
    (2, "Terrible... don't waste your money ", 1),
    (3, "Good quality, fast shipping!!!", 4),
    (4, "   not bad, could be better...   ", 3),
    (5, "AMAZING!!!!! Best purchase EVER!!!!!", 5),
    (6, "meh, it's OK I guess", 3),
    (7, "Product arrived damaged", 1),
    (8, "10/10 would recommend to friends!", 5),
]

df_reviews = spark.createDataFrame(reviews_data, ["id", "review", "rating"])
print("Raw reviews:")
df_reviews.show(truncate=False)

Raw reviews:
+---+---------------------------------------+------+
|id |review                                 |rating|
+---+---------------------------------------+------+
|1  |  GREAT product!!!  Would buy again.   |5     |
|2  |Terrible... don't waste your money     |1     |
|3  |Good quality, fast shipping!!!         |4     |
|4  |   not bad, could be better...         |3     |
|5  |AMAZING!!!!! Best purchase EVER!!!!!   |5     |
|6  |meh, it's OK I guess                   |3     |
|7  |Product arrived damaged                |1     |
|8  |10/10 would recommend to friends!      |5     |
+---+---------------------------------------+------+



In [20]:
def clean_text_pipeline(df, text_col, output_col="text_cleaned"):
    result = df \
        .withColumn("_step1_trim", trim(col(text_col))) \
        .withColumn("_step2_lower", lower(col("_step1_trim"))) \
        .withColumn("_step3_alphanum",
                    regexp_replace(col("_step2_lower"), "[^a-z0-9\\s]", "")) \
        .withColumn("_step4_spaces",
                    regexp_replace(col("_step3_alphanum"), "\\s+", " ")) \
        .withColumn(output_col, trim(col("_step4_spaces"))) \
        .drop("_step1_trim", "_step2_lower", "_step3_alphanum", "_step4_spaces")
    return result

df_reviews_cleaned = clean_text_pipeline(df_reviews, "review", "review_cleaned")

print("Cleaned reviews:")
df_reviews_cleaned.select("review", "review_cleaned").show(truncate=False)

Cleaned reviews:
+---------------------------------------+-------------------------------+
|review                                 |review_cleaned                 |
+---------------------------------------+-------------------------------+
|  GREAT product!!!  Would buy again.   |great product would buy again  |
|Terrible... don't waste your money     |terrible dont waste your money |
|Good quality, fast shipping!!!         |good quality fast shipping     |
|   not bad, could be better...         |not bad could be better        |
|AMAZING!!!!! Best purchase EVER!!!!!   |amazing best purchase ever     |
|meh, it's OK I guess                   |meh its ok i guess             |
|Product arrived damaged                |product arrived damaged        |
|10/10 would recommend to friends!      |1010 would recommend to friends|
+---------------------------------------+-------------------------------+



## 4.6 Practical Exercise: Product Review Cleaning

**Tasks:**
1. Clean the review text (remove special chars, normalize case, trim)
2. Extract the word count from each cleaned review
3. Identify reviews that mention negative keywords (return, refund, broken)
4. Calculate the average rating for reviews with negative keywords vs others

In [21]:
# Extended reviews dataset
extended_reviews = [
    (1, "LOVE IT! Fast shipping, great quality. Will order more!", 5),
    (2, "Product broke after 1 week. Requesting refund immediately.", 1),
    (3, "Decent product for the price. Nothing special but works.", 3),
    (4, "DO NOT BUY! Cheap quality, had to return it!", 1),
    (5, "Excellent! Better than expected. 100% recommend.", 5),
    (6, "Item was damaged during shipping. Broken on arrival.", 1),
    (7, "Good value for money. Satisfied with purchase.", 4),
    (8, "WORST PURCHASE EVER! Want my money back!!!", 1),
    (9, "Nice product, works as described. Happy customer.", 4),
    (10, "Meh, it's okay. Nothing to complain about.", 3),
]

df_extended = spark.createDataFrame(extended_reviews, ["id", "review", "rating"])
df_extended.show(truncate=False)

+---+----------------------------------------------------------+------+
|id |review                                                    |rating|
+---+----------------------------------------------------------+------+
|1  |LOVE IT! Fast shipping, great quality. Will order more!   |5     |
|2  |Product broke after 1 week. Requesting refund immediately.|1     |
|3  |Decent product for the price. Nothing special but works.  |3     |
|4  |DO NOT BUY! Cheap quality, had to return it!              |1     |
|5  |Excellent! Better than expected. 100% recommend.          |5     |
|6  |Item was damaged during shipping. Broken on arrival.      |1     |
|7  |Good value for money. Satisfied with purchase.            |4     |
|8  |WORST PURCHASE EVER! Want my money back!!!                |1     |
|9  |Nice product, works as described. Happy customer.         |4     |
|10 |Meh, it's okay. Nothing to complain about.                |3     |
+---+----------------------------------------------------------+

**Tasks:**
1. Clean the review text (remove special chars, normalize case, trim)
2. Extract the word count from each cleaned review
3. Identify reviews that mention negative keywords (return, refund, broken)
4. Calculate the average rating for reviews with negative keywords vs others

In [39]:
from pyspark.sql.functions import col, regexp_replace, lower, trim

cleaned_df = df_extended.withColumn(
    "step1_no_chars",
    # 1. Replace punctuation with a space (keeps words from sticking together)
    # We moved the hyphen to the end to avoid range errors
    regexp_replace(col("review"), r"[$&+,:;=?@#|'<>._^*()%!+-]", " ") 
).withColumn(
    "step2_single_space",
    # 2. Collapse "one or more" spaces (including the ones we just created) into ONE
    regexp_replace(col("step1_no_chars"), r"\s+", " ")
).withColumn(
    "cleaned_review",
    # 3. Final polish: Lowercase and Trim the very ends
    trim(lower(col("step2_single_space")))
).drop("step1_no_chars", "step2_single_space")

cleaned_df.show(truncate=False)

+---+----------------------------------------------------------+------+--------------------------------------------------------+
|id |review                                                    |rating|cleaned_review                                          |
+---+----------------------------------------------------------+------+--------------------------------------------------------+
|1  |LOVE IT! Fast shipping, great quality. Will order more!   |5     |love it fast shipping great quality will order more     |
|2  |Product broke after 1 week. Requesting refund immediately.|1     |product broke after 1 week requesting refund immediately|
|3  |Decent product for the price. Nothing special but works.  |3     |decent product for the price nothing special but works  |
|4  |DO NOT BUY! Cheap quality, had to return it!              |1     |do not buy cheap quality had to return it               |
|5  |Excellent! Better than expected. 100% recommend.          |5     |excellent better than expe

**Tasks:**
2. Extract the word count from each cleaned review

In [40]:
from pyspark.sql.functions import col, split, size

df_with_counts = cleaned_df.withColumn(
    "word_count",
    size(split(col("cleaned_review"), " "))
).withColumn(
    "word_array",
    split(trim(col("cleaned_review"))," ")
)

df_with_counts.select("cleaned_review", "word_count","word_array").show()

+--------------------+----------+--------------------+
|      cleaned_review|word_count|          word_array|
+--------------------+----------+--------------------+
|love it fast ship...|         9|[love, it, fast, ...|
|product broke aft...|         8|[product, broke, ...|
|decent product fo...|         9|[decent, product,...|
|do not buy cheap ...|         9|[do, not, buy, ch...|
|excellent better ...|         6|[excellent, bette...|
|item was damaged ...|         8|[item, was, damag...|
|good value for mo...|         7|[good, value, for...|
|worst purchase ev...|         7|[worst, purchase,...|
|nice product work...|         7|[nice, product, w...|
|meh it s okay not...|         8|[meh, it, s, okay...|
+--------------------+----------+--------------------+



In [43]:
df_exploed = df_with_counts.select(
    col("*"),
    explode(col("word_array")).alias("word"),
)

df_exploed.show(truncate=False)

+---+----------------------------------------------------------+------+--------------------------------------------------------+----------+-----------------------------------------------------------------+-----------+
|id |review                                                    |rating|cleaned_review                                          |word_count|word_array                                                       |word       |
+---+----------------------------------------------------------+------+--------------------------------------------------------+----------+-----------------------------------------------------------------+-----------+
|1  |LOVE IT! Fast shipping, great quality. Will order more!   |5     |love it fast shipping great quality will order more     |9         |[love, it, fast, shipping, great, quality, will, order, more]    |love       |
|1  |LOVE IT! Fast shipping, great quality. Will order more!   |5     |love it fast shipping great quality will order more     |

3. Identify reviews that mention negative keywords

In [44]:
df_with_counts.show(truncate=False)

+---+----------------------------------------------------------+------+--------------------------------------------------------+----------+-----------------------------------------------------------------+
|id |review                                                    |rating|cleaned_review                                          |word_count|word_array                                                       |
+---+----------------------------------------------------------+------+--------------------------------------------------------+----------+-----------------------------------------------------------------+
|1  |LOVE IT! Fast shipping, great quality. Will order more!   |5     |love it fast shipping great quality will order more     |9         |[love, it, fast, shipping, great, quality, will, order, more]    |
|2  |Product broke after 1 week. Requesting refund immediately.|1     |product broke after 1 week requesting refund immediately|8         |[product, broke, after, 1, week, requ

In [45]:
negative_keyword_array = ['refund','broke','return']

df_reviews_with_neg = df_with_counts.withColumn(
    "has_neg_word",
    col("cleaned_review").rlike("\\b(return|refund|broken|broke)\\b")
)
df_reviews_with_neg.show()

+---+--------------------+------+--------------------+----------+--------------------+------------+
| id|              review|rating|      cleaned_review|word_count|          word_array|has_neg_word|
+---+--------------------+------+--------------------+----------+--------------------+------------+
|  1|LOVE IT! Fast shi...|     5|love it fast ship...|         9|[love, it, fast, ...|       false|
|  2|Product broke aft...|     1|product broke aft...|         8|[product, broke, ...|        true|
|  3|Decent product fo...|     3|decent product fo...|         9|[decent, product,...|       false|
|  4|DO NOT BUY! Cheap...|     1|do not buy cheap ...|         9|[do, not, buy, ch...|        true|
|  5|Excellent! Better...|     5|excellent better ...|         6|[excellent, bette...|       false|
|  6|Item was damaged ...|     1|item was damaged ...|         8|[item, was, damag...|        true|
|  7|Good value for mo...|     4|good value for mo...|         7|[good, value, for...|       false|


 Task 4: Average rating for reviews with negative keywords vs others

In [47]:
df_review_cleaned = df_reviews_with_neg.drop("review","word_array")
df_review_cleaned.show(truncate=False)

+---+------+--------------------------------------------------------+----------+------------+
|id |rating|cleaned_review                                          |word_count|has_neg_word|
+---+------+--------------------------------------------------------+----------+------------+
|1  |5     |love it fast shipping great quality will order more     |9         |false       |
|2  |1     |product broke after 1 week requesting refund immediately|8         |true        |
|3  |3     |decent product for the price nothing special but works  |9         |false       |
|4  |1     |do not buy cheap quality had to return it               |9         |true        |
|5  |5     |excellent better than expected 100 recommend            |6         |false       |
|6  |1     |item was damaged during shipping broken on arrival      |8         |true        |
|7  |4     |good value for money satisfied with purchase            |7         |false       |
|8  |1     |worst purchase ever want my money back          

In [ ]:
df_reviews_avg = df_review_cleaned.groupBy("has_neg_word").agg(
    avg(col=col("rating")),
    count("*").alias("count")
)

df_reviews_avg.show(truncate=False)

+------------+------------------+-----+
|has_neg_word|avg(rating)       |count|
+------------+------------------+-----+
|true        |1.0               |3    |
|false       |3.5714285714285716|7    |
+------------+------------------+-----+



In [51]:
df_review_cleaned.createOrReplaceTempView("df_reviews")

df_reviews_avg = spark.sql("select count(has_neg_word),round(avg(rating),2),has_neg_word from df_reviews group by has_neg_word")
df_reviews_avg.show(truncate=False)

+-------------------+---------------------+------------+
|count(has_neg_word)|round(avg(rating), 2)|has_neg_word|
+-------------------+---------------------+------------+
|3                  |1.0                  |true        |
|7                  |3.57                 |false       |
+-------------------+---------------------+------------+



---
# Part 5: Regular Expressions in Spark
---

## 5.1 Introduction to Regex in PySpark

**Key Functions:**
- `rlike(pattern)` - Returns true if string matches pattern
- `regexp_extract(col, pattern, idx)` - Extracts groups matching the pattern
- `regexp_replace(col, pattern, replacement)` - Replaces matches with new text

**Common Regex Patterns:**
| Pattern | Meaning |
|---------|----------|
| `\d` | Any digit (0-9) |
| `\w` | Any word character |
| `\s` | Any whitespace |
| `.` | Any character |
| `*` | Zero or more |
| `+` | One or more |
| `[]` | Character class |
| `()` | Capture group |

In [52]:
# Sample data for regex operations
contact_data = [
    (1, "John Smith", "john.smith@email.com", "(555) 123-4567", "123 Main St, New York, NY 10001"),
    (2, "Jane Doe", "jane_doe@company.co.uk", "555-987-6543", "456 Oak Ave, Los Angeles, CA 90001"),
    (3, "Bob Wilson", "bob123@test.org", "5551234567", "789 Pine Rd, Chicago, IL 60601"),
    (4, "Alice Brown", "alice.brown@domain.net", "(555)456-7890", "321 Elm Blvd, Houston, TX 77001"),
    (5, "Charlie Lee", "invalid-email", "phone: 555-111-2222", "PO Box 100, Phoenix, AZ 85001"),
]

df_contacts = spark.createDataFrame(
    contact_data,
    ["id", "name", "email", "phone", "address"]
)

print("Contact data:")
df_contacts.show(truncate=False)

Contact data:
+---+-----------+----------------------+-------------------+----------------------------------+
|id |name       |email                 |phone              |address                           |
+---+-----------+----------------------+-------------------+----------------------------------+
|1  |John Smith |john.smith@email.com  |(555) 123-4567     |123 Main St, New York, NY 10001   |
|2  |Jane Doe   |jane_doe@company.co.uk|555-987-6543       |456 Oak Ave, Los Angeles, CA 90001|
|3  |Bob Wilson |bob123@test.org       |5551234567         |789 Pine Rd, Chicago, IL 60601    |
|4  |Alice Brown|alice.brown@domain.net|(555)456-7890      |321 Elm Blvd, Houston, TX 77001   |
|5  |Charlie Lee|invalid-email         |phone: 555-111-2222|PO Box 100, Phoenix, AZ 85001     |
+---+-----------+----------------------+-------------------+----------------------------------+



## 5.2 Pattern Matching with rlike

In [53]:
# Check if email is valid format
email_pattern = r"^[\w.+-]+@[\w-]+\.[a-zA-Z]{2,}$"

df_email_check = df_contacts.withColumn(
    "is_valid_email",
    col("email").rlike(email_pattern)
)

print("Email validation:")
df_email_check.select("name", "email", "is_valid_email").show(truncate=False)

Email validation:
+-----------+----------------------+--------------+
|name       |email                 |is_valid_email|
+-----------+----------------------+--------------+
|John Smith |john.smith@email.com  |true          |
|Jane Doe   |jane_doe@company.co.uk|false         |
|Bob Wilson |bob123@test.org       |true          |
|Alice Brown|alice.brown@domain.net|true          |
|Charlie Lee|invalid-email         |false         |
+-----------+----------------------+--------------+



In [54]:
# Filter for specific domain emails
df_company_emails = df_contacts.filter(
    col("email").rlike(r"@company\.")
)

print("Company domain emails:")
df_company_emails.select("name", "email").show()

Company domain emails:
+--------+--------------------+
|    name|               email|
+--------+--------------------+
|Jane Doe|jane_doe@company....|
+--------+--------------------+



## 5.3 Extracting Data with regexp_extract

In [55]:
# Extract email components
df_email_parts = df_contacts.withColumn(
    "email_username",
    regexp_extract(col("email"), r"^([\w.+-]+)@", 1)
).withColumn(
    "email_domain",
    regexp_extract(col("email"), r"@([\w-]+)\.", 1)
).withColumn(
    "email_tld",
    regexp_extract(col("email"), r"\.([a-zA-Z.]+)$", 1)
)

print("Extracted email components:")
df_email_parts.select("email", "email_username", "email_domain", "email_tld").show(truncate=False)

Extracted email components:
+----------------------+--------------+------------+---------+
|email                 |email_username|email_domain|email_tld|
+----------------------+--------------+------------+---------+
|john.smith@email.com  |john.smith    |email       |com      |
|jane_doe@company.co.uk|jane_doe      |company     |co.uk    |
|bob123@test.org       |bob123        |test        |org      |
|alice.brown@domain.net|alice.brown   |domain      |net      |
|invalid-email         |              |            |         |
+----------------------+--------------+------------+---------+



In [56]:
# Extract and normalize phone numbers
df_phone_extract = df_contacts.withColumn(
    "phone_digits",
    regexp_replace(col("phone"), r"[^\d]", "")
).withColumn(
    "area_code",
    regexp_extract(regexp_replace(col("phone"), r"[^\d]", ""), r"^(\d{3})", 1)
).withColumn(
    "phone_normalized",
    concat(
        lit("("),
        substring(regexp_replace(col("phone"), r"[^\d]", ""), 1, 3),
        lit(") "),
        substring(regexp_replace(col("phone"), r"[^\d]", ""), 4, 3),
        lit("-"),
        substring(regexp_replace(col("phone"), r"[^\d]", ""), 7, 4)
    )
)

print("Phone number extraction and normalization:")
df_phone_extract.select("phone", "phone_digits", "area_code", "phone_normalized").show(truncate=False)

Phone number extraction and normalization:
+-------------------+------------+---------+----------------+
|phone              |phone_digits|area_code|phone_normalized|
+-------------------+------------+---------+----------------+
|(555) 123-4567     |5551234567  |555      |(555) 123-4567  |
|555-987-6543       |5559876543  |555      |(555) 987-6543  |
|5551234567         |5551234567  |555      |(555) 123-4567  |
|(555)456-7890      |5554567890  |555      |(555) 456-7890  |
|phone: 555-111-2222|5551112222  |555      |(555) 111-2222  |
+-------------------+------------+---------+----------------+



In [57]:
# Extract address components
df_address_parts = df_contacts.withColumn(
    "street",
    regexp_extract(col("address"), r"^(.+),", 1)
).withColumn(
    "city",
    regexp_extract(col("address"), r", ([^,]+), [A-Z]{2}", 1)
).withColumn(
    "state",
    regexp_extract(col("address"), r", ([A-Z]{2}) \d", 1)
).withColumn(
    "zip_code",
    regexp_extract(col("address"), r"(\d{5})$", 1)
)

print("Extracted address components:")
df_address_parts.select("address", "street", "city", "state", "zip_code").show(truncate=False)

Extracted address components:
+----------------------------------+------------------------+-----------+-----+--------+
|address                           |street                  |city       |state|zip_code|
+----------------------------------+------------------------+-----------+-----+--------+
|123 Main St, New York, NY 10001   |123 Main St, New York   |New York   |NY   |10001   |
|456 Oak Ave, Los Angeles, CA 90001|456 Oak Ave, Los Angeles|Los Angeles|CA   |90001   |
|789 Pine Rd, Chicago, IL 60601    |789 Pine Rd, Chicago    |Chicago    |IL   |60601   |
|321 Elm Blvd, Houston, TX 77001   |321 Elm Blvd, Houston   |Houston    |TX   |77001   |
|PO Box 100, Phoenix, AZ 85001     |PO Box 100, Phoenix     |Phoenix    |AZ   |85001   |
+----------------------------------+------------------------+-----------+-----+--------+



## 5.4 Replacing Patterns with regexp_replace

In [58]:
# Mask sensitive data
df_masked = df_contacts.withColumn(
    "email_masked",
    regexp_replace(
        col("email"),
        r"^(.{2}).*(@.*)$",
        r"$1***$2"
    )
).withColumn(
    "phone_masked",
    regexp_replace(
        regexp_replace(col("phone"), r"[^\d]", ""),
        r"^(\d{3})(\d{3})(\d{4})$",
        r"(***) ***-$3"
    )
)

print("Masked sensitive data:")
df_masked.select("name", "email", "email_masked", "phone", "phone_masked").show(truncate=False)

Masked sensitive data:
+-----------+----------------------+-------------------+-------------------+--------------+
|name       |email                 |email_masked       |phone              |phone_masked  |
+-----------+----------------------+-------------------+-------------------+--------------+
|John Smith |john.smith@email.com  |jo***@email.com    |(555) 123-4567     |(***) ***-4567|
|Jane Doe   |jane_doe@company.co.uk|ja***@company.co.uk|555-987-6543       |(***) ***-6543|
|Bob Wilson |bob123@test.org       |bo***@test.org     |5551234567         |(***) ***-4567|
|Alice Brown|alice.brown@domain.net|al***@domain.net   |(555)456-7890      |(***) ***-7890|
|Charlie Lee|invalid-email         |invalid-email      |phone: 555-111-2222|(***) ***-2222|
+-----------+----------------------+-------------------+-------------------+--------------+



In [ ]:
# Clean and standardize text
messy_text_data = [
    (1, "Hello    World!!!"),
    (2, "Multiple   spaces   here"),
    (3, "Lots of!!!!! punctuation???"),
    (4, "   Leading and trailing   "),
    (5, "MixED CaSe TeXt"),
]

df_messy_text = spark.createDataFrame(messy_text_data, ["id", "text"])

df_standardized = df_messy_text.withColumn(
    "text_clean",
    trim(
        regexp_replace(
            regexp_replace(
                lower(col("text")),
                r"([!?.]){2,}",
                r"$1"
            ),
            r"\s+",
            " "
        )
    )
)

print("Text standardization:")
df_standardized.show(truncate=False)

## 5.5 Practical Exercise: Log File Parsing

Parse web server access logs using regex to extract structured information.

**Tasks:**
1. Extract IP address, timestamp, HTTP method, URL path, status code, and response size
2. Filter for error status codes (4xx and 5xx)
3. Count requests by HTTP method
4. Identify the most accessed URLs

In [ ]:
# Sample Apache access logs
log_data = [
    (1, ' '),
    (2, '10.0.0.50 - - [15/Jan/2024:10:31:00 +0000] "POST /api/users HTTP/1.1" 201 567'),
    (3, '192.168.1.101 - - [15/Jan/2024:10:31:15 +0000] "GET /about.html HTTP/1.1" 200 890'),
    (4, '172.16.0.25 - - [15/Jan/2024:10:31:30 +0000] "GET /missing.html HTTP/1.1" 404 234'),
    (5, '192.168.1.100 - - [15/Jan/2024:10:31:45 +0000] "GET /api/data HTTP/1.1" 500 100'),
    (6, '10.0.0.75 - - [15/Jan/2024:10:32:00 +0000] "PUT /api/users/1 HTTP/1.1" 200 456'),
    (7, '192.168.1.102 - - [15/Jan/2024:10:32:15 +0000] "DELETE /api/users/2 HTTP/1.1" 204 0'),
    (8, '172.16.0.30 - - [15/Jan/2024:10:32:30 +0000] "GET /index.html HTTP/1.1" 200 1234'),
    (9, '10.0.0.50 - - [15/Jan/2024:10:32:45 +0000] "POST /api/login HTTP/1.1" 401 89'),
    (10, '192.168.1.100 - - [15/Jan/2024:10:33:00 +0000] "GET /api/products HTTP/1.1" 200 5678'),
]

df_logs = spark.createDataFrame(log_data, ["id", "log_line"])
print("Sample access logs:")
df_logs.show(truncate=False)

Sample access logs:
+---+------------------------------------------------------------------------------------+
|id |log_line                                                                            |
+---+------------------------------------------------------------------------------------+
|1  |192.168.1.100 - - [15/Jan/2024:10:30:45 +0000] "GET /index.html HTTP/1.1" 200 1234  |
|2  |10.0.0.50 - - [15/Jan/2024:10:31:00 +0000] "POST /api/users HTTP/1.1" 201 567       |
|3  |192.168.1.101 - - [15/Jan/2024:10:31:15 +0000] "GET /about.html HTTP/1.1" 200 890   |
|4  |172.16.0.25 - - [15/Jan/2024:10:31:30 +0000] "GET /missing.html HTTP/1.1" 404 234   |
|5  |192.168.1.100 - - [15/Jan/2024:10:31:45 +0000] "GET /api/data HTTP/1.1" 500 100     |
|6  |10.0.0.75 - - [15/Jan/2024:10:32:00 +0000] "PUT /api/users/1 HTTP/1.1" 200 456      |
|7  |192.168.1.102 - - [15/Jan/2024:10:32:15 +0000] "DELETE /api/users/2 HTTP/1.1" 204 0 |
|8  |172.16.0.30 - - [15/Jan/2024:10:32:30 +0000] "GET /index.html HTT

extracting

In [98]:
# Your code here: Parse the log files
from pyspark.sql.functions import substring_index
df_extracted = df_logs.withColumn(
    "IP_Address",
    regexp_extract("log_line",r"^(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})",1)
).withColumn(
    "timeStamp",
    regexp_extract("log_line",r"\[[^\]]+\]",0)
).withColumn(
    "method",
    regexp_extract("log_line",r'\"(\w+)\s',1)
).withColumn(
    "path",
    regexp_extract("log_line",r'\"\w+\s(\S+)\s',1)
    ).withColumn(
    "status_code",
    trim(regexp_extract("log_line",r'\s(\d{1,3})\s',1)).cast(IntegerType())
).withColumn(
    "response_size",
    trim(regexp_extract("log_line",r'\s(\d+)$',1)).cast(IntegerType())
)
df_extracted.show(truncate=False)

+---+------------------------------------------------------------------------------------+-------------+----------------------------+------+-------------+-----------+-------------+
|id |log_line                                                                            |IP_Address   |timeStamp                   |method|path         |status_code|response_size|
+---+------------------------------------------------------------------------------------+-------------+----------------------------+------+-------------+-----------+-------------+
|1  |192.168.1.100 - - [15/Jan/2024:10:30:45 +0000] "GET /index.html HTTP/1.1" 200 1234  |192.168.1.100|[15/Jan/2024:10:30:45 +0000]|GET   |/index.html  |200        |1234         |
|2  |10.0.0.50 - - [15/Jan/2024:10:31:00 +0000] "POST /api/users HTTP/1.1" 201 567       |10.0.0.50    |[15/Jan/2024:10:31:00 +0000]|POST  |/api/users   |201        |567          |
|3  |192.168.1.101 - - [15/Jan/2024:10:31:15 +0000] "GET /about.html HTTP/1.1" 200 890   |192.1

count requests by HTTP method

In [99]:
df_extracted.groupBy("method").agg(count("*")).show()

+------+--------+
|method|count(1)|
+------+--------+
|  POST|       2|
|   GET|       6|
|DELETE|       1|
|   PUT|       1|
+------+--------+



In [100]:
df_extracted.groupBy("path").agg(count("*")).show()

+-------------+--------+
|         path|count(1)|
+-------------+--------+
|  /index.html|       2|
|/missing.html|       1|
|   /api/users|       1|
|  /about.html|       1|
|    /api/data|       1|
|/api/products|       1|
|   /api/login|       1|
| /api/users/2|       1|
| /api/users/1|       1|
+-------------+--------+



---
# Part 6: Building Complete Preprocessing Pipelines
---

## 6.1 Combining Multiple Preprocessing Steps

In [ ]:
# Real-world messy dataset
raw_data = [
    ("1", "  JOHN SMITH  ", "john.smith@EMAIL.com", "(555) 123-4567", "25", "$50,000.00", "2024-01-15"),
    ("2", "jane doe", "JANE_DOE@COMPANY.CO.UK", "555-987-6543", "N/A", "60000", "15/02/2024"),
    ("3", "Bob Wilson", "invalid-email", "5551234567", "35", "75,000", "March 20, 2024"),
    ("4", None, "alice@domain.net", "(555)456-7890", "28", None, "2024-04-10"),
    ("5", "Charlie Lee", "", "phone: 555-111-2222", "forty", "$80,000", None),
]

df_raw = spark.createDataFrame(
    raw_data,
    ["id", "name", "email", "phone", "age", "salary", "hire_date"]
)

print("Raw messy data:")
df_raw.show(truncate=False)

In [ ]:
def preprocess_employee_data(df):
    # Step 1: Clean and standardize name
    df = df.withColumn(
        "name_clean",
        when(col("name").isNull() | (trim(col("name")) == ""),
             concat(lit("Employee_"), col("id")))
        .otherwise(expr("initcap(trim(name))"))
    )
    # Step 2: Clean and validate email
    email_pattern = r"^[\w.+-]+@[\w-]+\.[a-zA-Z]{2,}$"
    df = df.withColumn(
        "email_clean",
        when(lower(trim(col("email"))).rlike(email_pattern),
             lower(trim(col("email"))))
        .otherwise(None)
    )
    # Step 3: Normalize phone number
    df = df.withColumn(
        "phone_digits",
        regexp_replace(col("phone"), r"[^\d]", "")
    ).withColumn(
        "phone_clean",
        when(length(col("phone_digits")) == 10,
             concat(
                 lit("("), substring(col("phone_digits"), 1, 3), lit(") "),
                 substring(col("phone_digits"), 4, 3), lit("-"),
                 substring(col("phone_digits"), 7, 4)
             ))
        .otherwise(None)
    )
    # Step 4: Convert age to integer
    df = df.withColumn("age_clean", col("age").cast(IntegerType()))
    # Step 5: Clean and convert salary
    df = df.withColumn(
        "salary_clean",
        regexp_replace(col("salary"), r"[$,]", "").cast(DoubleType())
    )
    # Step 6: Parse hire date
    df = df.withColumn(
        "hire_date_clean",
        coalesce(
            to_date(col("hire_date"), "yyyy-MM-dd"),
            to_date(col("hire_date"), "dd/MM/yyyy"),
            to_date(col("hire_date"), "MMMM dd, yyyy")
        )
    )
    return df.select(
        col("id").cast(IntegerType()).alias("id"),
        col("name_clean").alias("name"),
        col("email_clean").alias("email"),
        col("phone_clean").alias("phone"),
        col("age_clean").alias("age"),
        col("salary_clean").alias("salary"),
        col("hire_date_clean").alias("hire_date")
    )

df_clean = preprocess_employee_data(df_raw)

print("Cleaned data:")
df_clean.show(truncate=False)
df_clean.printSchema()

## 6.2 Data Quality Reporting

In [ ]:
def generate_quality_report(df_original, df_cleaned):
    total_rows = df_original.count()
    print("="*60)
    print("DATA QUALITY REPORT")
    print("="*60)
    print(f"\nTotal rows: {total_rows}")
    print("\n--- Missing Value Summary (Cleaned Data) ---")
    for col_name in df_cleaned.columns:
        null_count = df_cleaned.filter(col(col_name).isNull()).count()
        null_pct = (null_count / total_rows) * 100
        status = "OK" if null_pct < 10 else "WARNING" if null_pct < 30 else "CRITICAL"
        print(f"  {col_name}: {null_count} nulls ({null_pct:.1f}%) [{status}]")
    total_cells = total_rows * len(df_cleaned.columns)
    null_cells = sum(df_cleaned.filter(col(c).isNull()).count() for c in df_cleaned.columns)
    completeness = ((total_cells - null_cells) / total_cells) * 100
    print(f"\n--- Overall Data Quality Score ---")
    print(f"  Completeness: {completeness:.1f}%")
    print("="*60)

generate_quality_report(df_raw, df_clean)

## 6.3 Final Challenge: End-to-End Data Cleaning

Apply everything you've learned to clean a complex real-world dataset.

**Dataset:** E-commerce transaction data with multiple quality issues

**Tasks:**
1. Identify and report all data quality issues
2. Handle missing values appropriately for each column
3. Standardize text fields (product names, categories)
4. Parse and validate dates
5. Clean and validate email addresses
6. Extract useful information using regex
7. Create a comprehensive data quality report
8. Output the cleaned dataset

In [ ]:
# Complex e-commerce dataset
ecommerce_data = [
    ("ORD001", "  LAPTOP Computer  ", "ELECTRONICS", "john@email.com", "$999.99", "2", "2024-01-15 10:30:00", "Completed"),
    ("ORD002", "wireless mouse", "electronics", "JANE@COMPANY.CO.UK", "29.99", None, "15/01/2024", "completed"),
    ("ORD003", "USB-C Cable (6ft)", "Accessories", "invalid-email", "$12.50", "5", "January 16, 2024", "PENDING"),
    ("ORD004", None, "CLOTHING", "bob@test.org", "N/A", "1", "2024-01-16", "Cancelled"),
    ("ORD005", "Running Shoes - Size 10", "  clothing  ", "", "89.00", "2", None, "Shipped"),
    ("ORD006", "HDMI Cable!!!", "accessories", "alice@domain.net", "$15.00", "10", "2024-01-17 14:20:00", "completed"),
    ("ORD007", "smartphone case", "Electronics", "charlie@email.com", "$25", "3", "17-Jan-2024", "Processing"),
    ("ORD008", "  Winter Jacket  ", "Clothing", "N/A", "$149.99", None, "2024-01-18", "SHIPPED"),
]

df_ecommerce = spark.createDataFrame(
    ecommerce_data,
    ["order_id", "product", "category", "email", "price", "quantity", "order_date", "status"]
)

print("Raw e-commerce data:")
df_ecommerce.show(truncate=False)

In [ ]:
# Your code here: Build a complete preprocessing pipeline



---
# Cleanup
---

Always remember to stop your Spark session when you're done.

In [101]:
# Stop the Spark session
spark.stop()
print("Spark session stopped.")

Spark session stopped.


---
# Summary
---

In this lab, you learned:

1. **Missing Value Handling**
   - Identifying missing data (null, NaN, empty strings, placeholders)
   - Dropping rows/columns with missing values
   - Imputation strategies (mean, median, mode, group-wise)
   - Forward/backward fill for time series
   - Creating missing value indicators

2. **Data Type Management**
   - Understanding Spark data types
   - Type casting and conversion
   - Handling dates and timestamps
   - Schema validation and enforcement

3. **Text Preprocessing**
   - Case normalization and trimming
   - Tokenization and text splitting
   - Stop word removal
   - Building text cleaning pipelines

4. **Regular Expressions**
   - Pattern matching with rlike
   - Data extraction with regexp_extract
   - Pattern replacement with regexp_replace
   - Complex parsing tasks (emails, phones, addresses, logs)

5. **Building Pipelines**
   - Combining multiple preprocessing steps
   - Creating reusable functions
   - Data quality reporting

---
# Additional Resources
---

- Apache Spark Documentation: https://spark.apache.org/docs/latest/
- PySpark API Reference: https://spark.apache.org/docs/latest/api/python/
- Spark SQL Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
- Regular Expression Reference: https://docs.python.org/3/library/re.html
- Spark ML Feature Transformers: https://spark.apache.org/docs/latest/ml-features.html